In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep 
from selenium.webdriver.common.alert import Alert
import os
import subprocess
import csv
import pprint
import re
import pandas as panda

##### 設定ここから #####

START_STAFF_NAME = ''  ## 誰から始めるか（最初から始める場合は空文字にしておく）
END_STAFF_NAME = 'hoge'  ## 誰まで実行するか（最後まで実行する場合はhogeにしておく）
TARGET_STAFF_TYPE = '社員' ## 従業員種別（社員orアルバイト）

RESET_MODE = 0 ## 0の場合オフ、1の場合null以外のシフト登録削除、2の場合すべてのシフト登録削除
TEST_MODE = True ## True or False Trueの場合、確定ボタンは押さない

##### 設定ここまで #####

class JobCan:
    
    options = Options()
    options.add_argument('--disable-gpu');
    options.add_argument('--disable-extensions');
    options.add_argument('--proxy-server="direct://"');
    options.add_argument('--proxy-bypass-list=*');
    options.add_argument('--start-maximized');
    DRIVER_PATH = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chrome_options=options)
    
    target_course = ''
    target_area = ''
    target_staff_type = ''
    
    target_group_value = False
    
    ## csvの1行目を元に設定をする
    with open('ジョブカンスクリプト用 - CSV出力用.csv') as f:
        csv = list(csv.reader(f))
        for row in csv:
            if row[1] != "設定":
                break
            target_year = row[3]            ## 対象年
            target_month = row[5]         ## 対象月
            target_course = row[7]         ## 対象コース
            target_area = row[9]            ## 対象拠点
            target_staff_type = row[11]  ## 従業員種別（社員orアルバイト）
                
    def access_top(self):
        url = 'https://jobcan.ne.jp/'
        self.driver.get(url)
        
    def access_login(self):
        login_selector = '.l-header__login'
        element = self.driver.find_element_by_css_selector(login_selector)
        element.click()
        handle_array = self.driver.window_handles
        self.driver.switch_to.window(handle_array[-1])
    
    def confirm_administrator_account(self):
        
        form_company_selector = '#client_login_id'
        form_group_selector = '#client_manager_login_id'
        form_password_selector = '#client_login_password'

        form_company_element = self.driver.find_element_by_css_selector(form_company_selector)
        form_group_element = self.driver.find_element_by_css_selector(form_group_selector)
        form_password_element = self.driver.find_element_by_css_selector(form_password_selector)
        form_submit_element = self.driver.find_element_by_css_selector('button[type="submit"]')

        company_name = os.environ['JOBCAN_COMPANY_NAME']
        group_name = os.environ['JOBCAN_GROUP_NAME']
        password = os.environ['JOBCAN_PASSWORD']

        form_company_element.send_keys(company_name)
        form_group_element.send_keys(group_name)
        form_password_element.send_keys(password)
        form_submit_element.click()

    def hover_shift_manage_tab(self):
        self.wait_til_visibility('#manager_mypage_current_display')
        xpath = '//*[@id="header"]/div[2]/div/ul/li[2]/a'
        actions = ActionChains(self.driver)
        actions.move_to_element(self.driver.find_element_by_xpath(xpath)).perform()
        pallet_shift_month_element = self.driver.find_element_by_link_text('パレットシフト(月)')
        pallet_shift_month_element.click()
        
        self.driver.maximize_window()
        self.driver.execute_script("document.body.style.zoom='100%'")

    def input_shift_from_csv(self):
        with open('ジョブカンスクリプト用 - CSV出力用.csv') as f:
            shifts = list(csv.reader(f))
            start_index_now = False
            end_index_now = False
            print(f'{START_STAFF_NAME}さんから')
            print(f'{END_STAFF_NAME}さんまで登録します。')
            for shift in shifts:
                print("ーーーーーーーーーーーーーーーーーー")
                if(shift[0] == "FALSE"):  ## １列目がFALSEのとき、シフト登録対象ではないのでスキップ
                  continue
                print(f'{shift[1]}さんのシフトの登録を')
                if(START_STAFF_NAME in shift[1] and start_index_now == False):
                  start_index_now = True
                if(start_index_now == False):
                  print(f'スキップします')
                  continue
                print(f'開始します')
                self.open_pallet_shift(shift)
                if(END_STAFF_NAME in shift[1]):  ## END_STAFF_NAMEのシフト登録が完了したら終了
                  print(f'{shift[1]} == {END_STAFF_NAME} なので終了します')
                  break
    
    def open_pallet_shift(self, shift):
        staff_name = shift[1]
        
        form_target_year_element = self.driver.find_element_by_css_selector('[name="from[month][y]"]')
        form_target_year_element_select = Select(form_target_year_element)
        form_target_year_element_select.select_by_value(self.target_year)
        
        form_target_month_element = self.driver.find_element_by_css_selector('[name="from[month][m]"]')
        form_target_month_element_select = Select(form_target_month_element)
        form_target_month_element_select.select_by_value(self.target_month)
        
        form_target_group_element = self.driver.find_element_by_css_selector('[name="group_id"]')
        form_target_group_element_select = Select(form_target_group_element)
        
        ## グループのセレクトボックスを選択する。valueはtarget_group_select_value()で事前に用意している
        form_target_group_element_select.select_by_value(self.target_group_value)
        
        self.wait_til_visibility('#employee_finder_nametext')
        
        form_target_staff_name_element = self.driver.find_element_by_css_selector('#employee_finder_nametext')
        sleep(0.1)
        form_target_staff_name_element.clear()
    
        action = ActionChains(self.driver)
        action.move_to_element(form_target_staff_name_element).click().send_keys(staff_name).perform()
        search_results = self.driver.find_element_by_css_selector('.tt-dataset-employees')
        if("候補なし" in search_results.text):  ## 候補なしが表示された場合、グループにいないスタッフなので次のスタッフの登録へ移る
            form_tags_element = self.driver.find_element_by_css_selector('#tags')
            action.move_to_element(form_tags_element).click().perform()
            form_target_staff_name_element.clear()
            return "false"
        
        ## 表示ボタンをクリックしてパレットを表示する
        form_submit_element = self.driver.find_element_by_link_text('表示')
        form_submit_element.click()
        self.alert_accept(message = '候補あり〜表示ボタンクリック')
        self.wait_til_visibility('#employee_finder_nametext')
        self.driver.find_element_by_tag_name('body').send_keys(Keys.END)
            
        ## シフトをパレットへ登録開始
        self.input_shift_to_pallete(shift[15:45])
        
    def target_group_select_value(self):  ## グループの選択肢のvalueを取得する
        target_groups = self.driver.find_elements_by_xpath(f"//option[contains(text(), '{self.target_course}')]")
        target_group_value = False
        for element in target_groups :      
            if self.target_area in element.text and TARGET_STAFF_TYPE in element.text:
                self.target_group_value = element.get_attribute("value")
                break

    def input_shift_to_pallete(self, shift):
        self.wait_til_visibility('tr[id*="shift-definite-list"]')
        print('★★★シフト入力開始！★★★')
        pallete_tr_element = self.driver.find_element_by_css_selector('tr[id*="shift-definite-list"]')
        shift_box_elements = self.driver.find_elements_by_css_selector('span[id*="shift-definite-data"]')
        changed_count = 0
        action = ActionChains(self.driver)
        
        ## シフトパターンやシフトパレットがちょうど良く見える位置にスクロールする
        ## clickする要素が画面外にあると失敗するため
        center_element = self.driver.find_element_by_css_selector('#form-table')
        action.move_to_element(center_element).perform()
        
        for shift_box_element, shift_data in zip(shift_box_elements, shift):
            ## リセットモード　0の場合オフ、1の場合null以外のシフト登録削除、2の場合すべてのシフト登録削除
            if shift_data == "null" and RESET_MODE != 2 :  ## シフトデータがnullなのでスキップ
                continue
            if  "総会" in shift_data:
                shift_data = "9-18"
            shift_pattern = self.convert_shift_data_to_shift_pattern(shift_data)
            print(f"before=>{shift_box_element.text}     after=>{shift_pattern}({shift_data})")
            if RESET_MODE == 1 or RESET_MODE == 2 :   ## リセットモードによりシフトを削除
                changed_count += 1
                shift_pattern_button = self.driver.find_element_by_css_selector(f'#elem-of-none')
            elif shift_pattern == "その他" :  ## 時間指定でシフトを登録
                changed_count += 1
                self.input_other_shift_to_pallete(shift_data, shift_box_element)
                continue
            else :  ## 遅番や中抜けなどのパターンでシフトを登録
                changed_count += 1
                shift_pattern = self.convert_shift_data_to_shift_pattern(shift_data)
                self.wait_til_visibility('[value="変更を保存する"]')
                shift_pattern_button = self.driver.find_element_by_css_selector(f'.pallet-element > input[value*="{shift_pattern}"]').find_element_by_xpath('..')
            ## シフトパターンボタンをクリック
            shift_pattern_button.click()
            ## 公or有がセットされているシフトを編集しようとするとアラートが出るのでアラートを承認
            if '公' in shift_box_element.text :
                shift_box_element.click()
                self.alert_accept(message = '公のシフトを編集しようとした')
            elif  '有' in shift_box_element.text:
                print('【注意】有のシフトを編集しようとしたのでスキップします【注意】')
            else:
                shift_box_element.click()
            ## シフトボックスに公休が設定されている場合一旦消す
            if '公' in shift_box_element.text and not '休' in shift_box_element.text:
                holiday_pattern_element = self.driver.find_element_by_css_selector('#elem-of-public_holiday')
                holiday_pattern_element.click()
                shift_box_element.click()
                
        ## シフトを確定する
        if TEST_MODE == True:
            self.alert_accept(message = 'テストモードにより確定をスキップ...')
        else :
            ## 確定ボタン取得
            confirm_button = self.driver.find_element_by_css_selector('[value="変更を保存する"]')
            ## 確定ボタンをクリック
            action.move_to_element(confirm_button).click().perform()
    #         confirm_button.click()
            self.alert_accept(message = 'シフト確定ボタンを押した')
            if changed_count != 0:
                self.alert_accept(wait_seconds = 5, retry = True, message = 'シフト確定ボタンを押してから完了した')
            
    def convert_shift_data_to_shift_pattern(self, shift_data):  ## csvから読み取ったシフト情報を遅番や中抜けなどに変換する。例えば10-19は早Aになる
        if shift_data in "10-19" :
            shift_pattern = "早A"
        elif shift_data in "11-20" :
            shift_pattern = "早B"
        elif shift_data in "14-22" :
            shift_pattern = "遅番"
        elif shift_data in "中11-22" :
            shift_pattern = "中抜"
        elif shift_data in "外A" :
            shift_pattern = "外A"
        elif shift_data in "外B" :
            shift_pattern = "外B"
        elif shift_data in "外C" :
            shift_pattern = "外C"
        elif shift_data in "休" :
            shift_pattern = "休"
        else:
            shift_pattern = "その他"
        return shift_pattern
    
    def input_other_shift_to_pallete(self, shift_data, shift_box_element):  ## 時間指定でシフトをセットする
        shift_times = self.convert_time_from_shift_data(shift_data)
        other_shift_button_element = self.driver.find_element_by_css_selector('#line_shift_div > .pallet-element')
        other_shift_button_element.click()
        shift_box_element.click()
        sleep(0.5)
        self.select_other_shift(shift_times)
        apply_button_element = self.driver.find_element_by_link_text('適用')
        apply_button_element.click()
        self.alert_accept(message = '時間指定シフトを登録した')
    
    def alert_accept(self, wait_seconds = 0.5, retry = False, message = ""):  ## アラートが出るのを待機して承認する
        if message != "" :
            print(f"アラート待機：{message}")
        try:
            WebDriverWait(self.driver, wait_seconds).until(EC.alert_is_present(),
                                           'Timed out waiting for PA creation ' +
                                           'confirmation popup to appear.')

            alert = self.driver.switch_to.alert
            alert.accept()
            print("◯アラートを承認しました")
        except TimeoutException:
            print("×アラートが捕捉できませんでした")
            if retry :
                print("再試行します")
                self.alert_accept(wait_seconds = wait_seconds, message = message + "：再試行")
            
    def convert_time_from_shift_data(self, shift_data):  ## シフトデータ（例14.5-17）を変換してハッシュにまとめて返す
        ## 例えば14.5-17は　{start_hour: 14, start_minutes:30, end_hour: 17, end_minutes:0}　になる
        start_minutes = 0
        end_minutes = 0
        splited_data= shift_data.split('-')  ## 14.5-17を ['14.5', ''17']に分解する
        start_hour = splited_data[0]  ## ['14.5', ''17']のうち14.5を取得
        end_hour = splited_data[1]   ## ['14.5', ''17']のうち17を取得

        if('.' in start_hour):  ## 14.5のように小数が含まれるときは分の算出をする
            start_minutes = self.convert_minutes_from_hour(start_hour)  ##  分を算出する'例えば14.5が渡されたら30を返す'
        if('.' in end_hour):  ## 14.5のように小数が含まれるときは分の算出をする
            end_minutes = self.convert_minutes_from_hour(end_hour)

        start_hour = self.convert_hour_from_string(start_hour)  ## 例えば'14.5'を14に変換する
        end_hour = self.convert_hour_from_string(end_hour)
        
        shift_times = {'start_hour': start_hour, 'start_minutes': start_minutes, 'end_hour': end_hour, 'end_minutes': end_minutes }
#         rest_times = self.calc_rest_times_from_shift_times(shift_times)
        return shift_times

    def convert_hour_from_string(self, hour_string):  ##  シフト時間から時間を算出する。例えば'14.5'が渡されたら14を返す'
        hour_replaced = hour_string.replace('新', '')
        hour_float = float(hour_replaced)
        hour_int = int(hour_float / 1.0)
        return hour_int

    def convert_minutes_from_hour(self, hour_string): ##  シフト時間から分を算出する。例えば'14.5'が渡されたら0.5を返す'
        hour_replaced = hour_string.replace('新', '')
        hour_to_float = float(hour_replaced)  ## 文字列'14.5' を数値14.5に変換する
        decimal_ratio_float = hour_to_float % 1 ## 14.5を0.5に変換する
        minutes_float = 60 * decimal_ratio_float  ## 60分に倍率（0.5）をかける
        minutes_int = int(minutes_float)
        return minutes_int
    
    def select_other_shift(self, shift_times):
        ## ジョブカンの時間のセレクトボックスのvalueは「実際の時間-5」、例えば14時の場合は9になっている
        self.set_value_to_select_form('[name="lstart_h"]', shift_times['start_hour'] -5)
        self.set_value_to_select_form('[name="lstart_m"]', shift_times['start_minutes'])
        self.set_value_to_select_form('[name="lend_h"]', shift_times['end_hour'] -5)
        self.set_value_to_select_form('[name="lend_m"]', shift_times['end_minutes'])
        
    def set_value_to_select_form(self, selector, value):  ## セレクトボックスを選択する
        self.wait_til_visibility(selector)
        form_target_element = self.driver.find_element_by_css_selector(selector)
        form_target_element_select = Select(form_target_element)
        form_target_element_select.select_by_value(str(value))
        
    def wait_til_visibility(self, selector):
        document_load_element = WebDriverWait(self.driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, selector))
        )

jobcan = JobCan()
jobcan.access_top()
jobcan.access_login()
jobcan.wait_til_visibility('.login-title')
jobcan.confirm_administrator_account()
jobcan.hover_shift_manage_tab()
jobcan.target_group_select_value()
if not jobcan.target_group_value :
    print('【エラー】対象グループが正しくありません。')
else :
    jobcan.input_shift_from_csv()
    print('★★★登録が完了しました★★★')
